In [1]:
import json
from urllib.request import urlopen
import requests
import pandas as pd

In [2]:
def get_jsonparsed_data(projectId):
    response = requests.get("https://tasks.hotosm.org/api/v1/project/" + str(projectId))
    return response.json()

def read_jsonfile_data(fileName):
    with open(fileName) as f:
        data = json.load(f)
    return data    
    
projects = []
project_json = read_jsonfile_data("project.json")

In [3]:
result = pd.DataFrame(columns=['ProjectId', 'TaskId', 'TaskStatus', 'Coordinates'])

In [7]:
validated_percent = {}

In [18]:
for key,value in enumerate(project_json['mapResults']['features']):
    projects.append(value['properties']['projectId'])
    
for projectId in projects:
    val = 0
    task_json = get_jsonparsed_data(projectId)
    for task in task_json['tasks']['features']:
        #result = result.append({'ProjectId' : projectId,
        #               'TaskId' : task['properties']['taskId'],
        #               'TaskStatus' : task['properties']['taskStatus'],
        #               'Coordinates' : task['geometry']['coordinates']}, ignore_index = True)
        if task['properties']['taskStatus'] == "VALIDATED":
            val += 1
    validated_percent[projectId] = val / len(task_json['tasks']['features'])
  

In [51]:
np.set_printoptions(precision=0,suppress=True) 
final =  [int(k) for k, v in validated_percent.items() if v > 0.90]

In [52]:
 final_1 = [int(k) for k, v in validated_percent.items() if v == 1]

In [53]:
import numpy as np
a = np.asarray(final_1)
np.savetxt("project_100b.csv", a, delimiter=",")

In [54]:
a = np.asarray(final)
np.savetxt("project_90b.csv", a, delimiter=",")